In [3]:
import numpy as np
import pandas as pd

from scipy import stats, special, spatial

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import importlib.util
import os

import warnings
warnings.filterwarnings('ignore')

#s.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import keras
import numpy as np
import pandas as pd
import importlib.util
import os

from scipy import stats, special, spatial
from scipy.stats import gaussian_kde

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
import matplotlib as mpl

# https://github.com/jnothman/UpSetPlot
import upsetplot

import warnings
warnings.filterwarnings('ignore')

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [4]:
#Datasets
#Upregulator
AscomycotaUpXval = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_permutate/top_features_permute_final.npy', allow_pickle=True)
AscomycotaUpXhot = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_permutate/top_permutate.Xhot.npy', allow_pickle=True)
UpCombined = [AscomycotaUpXhot, AscomycotaUpXval]
#Downregulator
AscomycotaDownXval = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_permutate/bottom_features_permute_final.npy', allow_pickle=True)
AscomycotaDownXhot = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_permutate/bottom_permutate.Xhot.npy', allow_pickle=True)
DownCombined = [AscomycotaDownXhot, AscomycotaDownXval]

## datasets
folder = 'D:/Users/ernes/Documents/1KCL/Zelezniak/Data1/'

#Model
fname_data1 = folder+'scerevisiae.rsd1.lmbda_22.npz'
fname_module1 = folder+'Model_C3F2all.py'
fname_p1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best.p'
fname_weights1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best'

fname_module2 = folder+'Model_L2_C3F2addvarsopen_loadox.py'
fname_data2 = folder+'scerevisiae.rsd1.lmbda_22.1000.npz'
fname_p2 = folder+'scerevisiae_rsd1_merged_Model_C3F2_open_1241_0c6919106ab536108b55fd92965886b3_best.p'
fname_weights2 = folder+'scerevisiae_rsd1_merged_Model_C3F2_open_1241_0c6919106ab536108b55fd92965886b3_best'

In [5]:
def load_data(fname):
    #X is multi-variable array
    #Y contains single variable - fix shape for Keras

    npzfile = np.load(fname,allow_pickle=True)
    Xh_train = npzfile['arr_0']
    Xh_test = npzfile['arr_1']
    Xv_train = npzfile['arr_2'][:,8:]
    Xv_test = npzfile['arr_3'][:,8:]
    Y_train = npzfile['arr_4']
    Y_test = npzfile['arr_5']
    names_train = npzfile['arr_6']
    names_test = npzfile['arr_7']

    X_train = list()
    X_train.append(Xh_train)
    #X_train.append(Xv_train)
    X_test = list()
    X_test.append(Xh_test)
    #X_test.append(Xv_test)

    Y_train = Y_train.astype(np.float32).reshape((-1,))
    Y_test = Y_test.astype(np.float32).reshape((-1,))

    return (np.concatenate((Y_train, Y_test)), 
            np.concatenate((names_train,names_test)), 
            np.concatenate((Xv_train,Xv_test)))

In [6]:
#load modules - POC model
def load_module(model_path):
    '''loads module containing models given path'''
    spec = importlib.util.spec_from_file_location('module',model_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

In [7]:
#Load data and model
module = load_module(fname_module1)

#Load p
p = pd.read_csv(fname_p1,sep=';')
p = p.iloc[0]
p.drop(['path','overfit'],inplace=True)
print(p.shape)

#Load data
X_train, X_test, Y_train, Y_test = module.load_data(fname_data1) 
# Y = np.concatenate((Y_train, Y_test))
# X = np.concatenate((X_train[0], X_test[0]))
print(len(X_test))
print(len(Y_test))

#Model
input_shapes = [sl.shape[1:] for sl in X_train]
model = module.POC_model(input_shapes, p)
#Set weights model 1
print("Loading model 1 from disk..")
model.load_weights(fname_weights1)

(42,)
2
425
Loading model 1 from disk..


In [8]:
#Predict 
# Predicting on each 
Permutated_Up_Pred = model.predict(UpCombined)
Permutated_Down_Pred = model.predict(DownCombined)

154/154 [==============================] - 17s 108ms/step


In [11]:
file_path = r"D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_permutate/Permutated_Up_Pred.npy"
#Save the array to the specified file
np.save(file_path, Permutated_Up_Pred)

file_path = r"D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_permutate/Permutated_Down_Pred.npy"
#Save the array to the specified file
np.save(file_path, Permutated_Down_Pred)